# <span> Module 4: Running the module with new data <span>

In this notebook, we are going to explore how to run this module with a new dataset. These submodules provide a great framework for running a rigorous and scalable analysis, but there are some considerations that must be made in order to run this with your own data. We will walk through that process here so that hopefully, you are able to take these notebooks to your research group and use them for your own analysis. Notice that we do not give you all the answers in the code blocks, but if you get stuck, use the dropdowns for help. This module largely uses Nextflow for the RNA-seq and Methyl-seq analysis, which makes it very easy to run the same analysis on new datasets by updating the config files.

## **Importing the example dataset**

Our new dataset comes from a paper by [Hadad et al. Epigenetics Chromatin. 2019](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6781367/) that compares methylation changes in mice as they age and correlates those changes to gene expression changes. The data is available in SRA under the bioProject number [PRJNA523985](https://www.ncbi.nlm.nih.gov/sra/?term=PRJNA523985). The impact of methylation on aging, particularly in brain tissue, is of great research interest. There are many samples in this dataset, but we will limit our analysis to young vs old female mice.

To download the dataset, follow the instructions in the [STRIDES tutorial](https://github.com/STRIDES/NIHCloudLabGCP/blob/main/tutorials/notebooks/SRADownload/SRA-Download.ipynb) on downloading datasets from SRA using the prefetch+fasterq dump method. The accession numbers for this analysis are: 

- SRR8616802
- SRR8616795
- SRR8616796
- SRR8616777
- SRR8616778
- SRR8616772
- SRR8616799
- SRR8616800
- SRR8616801
- SRR8616787
- SRR8616788
- SRR8616789

You can save these accession numbers in a file and use that in the `sra-tools` commands. Once you pull these files from SRA, store them in a storage bucket so that Nextflow can see them in the next steps.

## **RNA-Seq analysis**

To run the RNA-Seq portion of this tutorial, you need to update the config file to point to your RNA-Seq reads. Let's look at the `rnaseq-gcb.config` file and make the necessary changes. We will need to specify `params.outdir`, `workDir`, `params.input`, and `params.genome`.

```
profiles {
  gcb {
      // Google batch parameters
      process.executor = 'google-batch'
      process.container = 'quay.io/nextflow/rnaseq-nf:v1.1'
      google.location = 'us-central1'
      google.region  = 'us-central1'
      google.project = '<Project ID>'
      // Workflow parameters
      params.outdir = ''
      workDir = ''
      params.input = ''
      params.genome = ''
  }
}
```

Check the [nf-core RNA-seq](https://nf-co.re/rnaseq/3.12.0) documentation to find out how most of this is structured. As with the primary dataset, the workDir and outdir are going to be locations in your Google Cloud storage bucket. The sequences are from mice so we need to specify a mouse genome build. Finally, the most effort goes into the input parameter. The nf-core documentation specifies that the reads need to be structured in a sample sheet. Let's look at an example. Using your bucket paths to your reads, create a .csv like this and put it in your bucket, then provide the bucket path in the config file. If you need help, click the help button below to see how we suggest.

<details>
  <summary>Click for help</summary>

```
params.outdir = 'gs://PROJECT-ID/rnaseq/results'
workDir = 'gs://PROJECT-ID/rnaseq/work'
params.input = 'gs://PROJECT-ID/samplesheet.csv'
params.genome = 'GRCm38'
```

</details>


Here's how the first row of your sample sheet might look. Make sure to only include the RNA-seq samples in the sample sheet. The methyl-seq samples will be included in that sample sheet when we run that pipeline in the next step.

```   
sample,fastq_1,fastq_2,strandedness   
Young_1,gs://BUCKETPATH/SRR8616802_1.fastq.gz,gs://BUCKETPATH/SRR8616802_2.fastq.gz,auto
```   

Once you have that together, run the Nextflow command again to run the nf-core pipeline on this dataset. 

After the Nextflow quantification completes, you can follow much of the RNA-seq notebook as written using the `salmon.merged.gene_counts.tsv` file that is produced by this step. You will need to update the `sample-info.txt` file to match the new information from these samples, but that is a simple task. As you go through the analysis, think about what comparisons would be interesting to make. Feel free to consult the manuscript for analysis ideas and try to replicate some of their results. 

## **DNA Methylation Analysis**

We will use the same framework for the methylation analysis as for the RNA-seq, which is to adjust the config file and let Nextflow run the pipeline. Let's look at the methylation config file and determine what we need to change. Like before, we need to specify a sample sheet input, the genome, workdir, and outdir. Fill in those below and try running the Nextflow command to run the core methyl-seq analysis.

```
profiles {
  gcb {
      // Google batch parameters
      process.executor = 'google-batch'
      process.container = 'quay.io/nextflow/rnaseq-nf:v1.1'
      google.project = 'PROJECT-ID'
      google.location = 'us-central1'
      google.region  = 'us-central1'
      process.machineType = 'c2-standard-30'
      // Workflow parameters
      dag.overwrite = true
      params.outdir = 'FILL-IN-HERE'
      workDir = 'FILL-IN-HERE'
      params.genome = `FILL-IN-HERE`
      params.input = `FILL-IN-HERE`
  }
}
```

Be sure to consult the [nf-core methylseq documentation](https://nf-co.re/methylseq/2.5.0) to see how the input sample sheet is structured. There are some differences from the RNA-seq input. Here's an example of our suggested first line.

```
sample,fastq_1,fastq_2   
Young_1,gs://BUCKETPATH/SRR8616795_1.fastq.gz,gs://BUCKETPATH/SRR8616795_2.fastq.gz
```

As with the RNA-seq portion of this notebook, much of the remaining analysis can be completed as written in the RRBS-downstream.ipynb notebook. You will need to update the cells that hard-code sample and file information to match the samples from the new dataset, but this is a straightforward task. After doing this, try working through the Integration.ipynb notebook with your new 